In [1]:
import datetime as dt
import pandas as pd

In [2]:
# identifier for each team on pro-football-reference
teams = ['crd', 'atl', 'rav', 'buf', 
         'car', 'chi', 'cin', 'cle', 
         'dal', 'den', 'det', 'gnb', 
         'htx', 'clt', 'jax', 'kan', 
         'sdg', 'ram', 'mia', 'min', 
         'nwe', 'nor', 'nyg', 'nyj', 
         'rai', 'phi', 'pit', 'sfo', 
         'sea', 'tam', 'oti', 'was']
print(len(teams))

32


In [3]:
# year range starts in 2002 because the houston texans didn't exist in a prior form, Causing it to kick an error later in the program.
current_season = dt.date.today().year
season_range = range(2002, current_season + 1)
num_seasons = (season_range[-1] - season_range[0] + 1)
season_dict = {}

In [4]:
%%time
# general data ingestion for every year, every team
# iterates through pro-football-reference's website, pulling the season breakdown table out of the html on each page.
# note that it doesn't pull in the current season's information. That will happen in the next stage after generating historical ELOs

progress_counter = 1
for season in season_range:
    iter_dict={}
    for team in teams:
        url = f'https://www.pro-football-reference.com/teams/{team}/{season}.htm'
        data = pd.read_html(url, header=1)
        iter_dict[team] = data[1]
        iter_dict[team]['season'] = season # attaches what season the data is from
        iter_dict[team]['year'] = season #attaches year column, to be incremented by one for games in Jan/Feb
        iter_dict[team]['team'] = team # attaches what team
    season_dict[season] = iter_dict
    print(f'{season} | {progress_counter} / {num_seasons} COMPLETE')
    progress_counter += 1
    print('-------------')

2002 | 1 / 19 COMPLETE
-------------
2003 | 2 / 19 COMPLETE
-------------
2004 | 3 / 19 COMPLETE
-------------
2005 | 4 / 19 COMPLETE
-------------
2006 | 5 / 19 COMPLETE
-------------
2007 | 6 / 19 COMPLETE
-------------
2008 | 7 / 19 COMPLETE
-------------
2009 | 8 / 19 COMPLETE
-------------
2010 | 9 / 19 COMPLETE
-------------
2011 | 10 / 19 COMPLETE
-------------
2012 | 11 / 19 COMPLETE
-------------
2013 | 12 / 19 COMPLETE
-------------
2014 | 13 / 19 COMPLETE
-------------
2015 | 14 / 19 COMPLETE
-------------
2016 | 15 / 19 COMPLETE
-------------
2017 | 16 / 19 COMPLETE
-------------
2018 | 17 / 19 COMPLETE
-------------
2019 | 18 / 19 COMPLETE
-------------
2020 | 19 / 19 COMPLETE
-------------
Wall time: 6min 27s


In [5]:
# testing that I can access the dataframe as expected
season_dict[2002]['chi'].head()

,Week,Day,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,OT,Rec,Unnamed: 8,Opp,...,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,season,year,team
0,1,Sun,September 8,1:04PM ET,boxscore,W,NaN,1-0,NaN,Minnesota Vikings,...,368.0,228.0,140.0,3.0,6.37,-1.20,1.01,2002,2002,chi
1,2,Sun,September 15,1:04PM ET,boxscore,W,NaN,2-0,@,Atlanta Falcons,...,257.0,135.0,122.0,3.0,-9.67,8.39,3.54,2002,2002,chi
2,3,Sun,September 22,1:05PM ET,boxscore,L,NaN,2-1,NaN,New Orleans Saints,...,302.0,229.0,73.0,3.0,0.58,-0.64,-6.03,2002,2002,chi
3,4,Sun,September 29,1:04PM ET,boxscore,L,OT,2-2,@,Buffalo Bills,...,410.0,307.0,103.0,1.0,3.14,-7.40,0.06,2002,2002,chi
4,5,Mon,October 7,9:08PM ET,boxscore,L,NaN,2-3,NaN,Green Bay Packers,...,457.0,333.0,124.0,1.0,-12.87,-3.38,8.55,2002,2002,chi


In [6]:
# Dictionaries for cleaning up data

# column cleaning dictionary
column_clean_dict =  {
     'Day': 'day_week',
     'Date' : 'date',
     'Week' : 'week',
     'Unnamed: 3':'kickoff_time',
     'Opp' : 'opp',
     'OT' : 'ot',
     'Rec' : 'record',
     'Unnamed: 5': 'W/L',
     'Unnamed: 8' : 'home/away',
     'Tm' : 'team_score',
     'Opp.1' : 'opp_score',
     '1stD' : '1D_gained',
     'TotYd' : 'team_total_yards',
     'PassY' : 'team_pass_yards',
     'RushY' : 'team_rush_yards',
     'TO' : 'team_turnover',
     '1stD.1' : '1D_allowed',
     'TotYd.1' : 'opp_total_yards',
     'PassY.1' : 'opp_pass_yards',
     'RushY.1' : 'opp_rush_yards',
     'TO.1' : 'opp_turnover',
     'Offense' : 'offense_expected_points',
     'Defense' : 'defense_expected_points',
     'Sp. Tms' : 'sp_expected_points'
    }

# Column Order
column_ord = ['team', 'season', 'opp', 'datetime',
              'day_week', 'week', 'W/L', 'ot', 'record', 'home/away', 
              'team_score', 'opp_score', '1D_gained', 'team_total_yards',
              'team_pass_yards', 'team_rush_yards', 'team_turnover', '1D_allowed',
              'opp_total_yards','opp_pass_yards', 'opp_rush_yards', 'opp_turnover',
              'offense_expected_points', 'defense_expected_points', 'sp_expected_points']

# dictionary for mapping month into form for datetime transformation
month_dict = {
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12,
    'January' : 1,
    'February': 2
    }

# dictionary to turn team id to standard three letter
acro_dict = {
    'crd' : 'ARI', 'atl' : 'ATL','rav' : 'BAL','buf' : 'BUF', 
    'car' : 'CAR', 'chi' : 'CHI', 'cin' : 'CIN', 'cle' : 'CLE', 
    'dal' : 'DAL', 'den' : 'DEN', 'det' : 'DET', 'gnb' : 'GNB', 
    'htx' : 'HOU', 'clt' : 'IND', 'jax' : 'JAX', 'kan' : 'KAN', 
    'sdg' : 'LAC', 'ram' : 'LAR', 'mia' : 'MIA', 'min' : 'MIN', 
    'nwe' : 'NWE', 'nor' : 'NOR', 'nyg' : 'NYG', 'nyj' : 'NYJ', 
    'rai' : 'LVR', 'phi' : 'PHI', 'pit' : 'PIT', 'sfo' : 'SFO', 
    'sea' : 'SEA', 'tam' : 'TAM', 'oti' : 'TEN', 'was' : 'WAS'
    }

# dictionary to edit 'opp' column
# the reason this one is a little longer is due to some teams changing cities, leading to the opp column to record their full name at the time.
opp_dict = {
    'Arizona Cardinals' : 'ARI', 'Atlanta Falcons' : 'ATL',
    'Baltimore Ravens' : 'BAL','Buffalo Bills' : 'BUF', 
    'Carolina Panthers' : 'CAR', 'Chicago Bears' : 'CHI', 
    'Cincinnati Bengals' : 'CIN', 'Cleveland Browns' : 'CLE', 
    'Dallas Cowboys' : 'DAL', 'Denver Broncos' : 'DEN', 
    'Detroit Lions' : 'DET', 'Green Bay Packers' : 'GNB', 
    'Houston Texans' : 'HOU', 'Indianapolis Colts' : 'IND', 
    'Jacksonville Jaguars' : 'JAX', 'Kansas City Chiefs' : 'KAN', 
    'Los Angeles Chargers' : 'LAC', 'San Diego Chargers' : 'LAC',
    'Los Angeles Rams' : 'LAR', 'St. Louis Rams': 'LAR',
    'Miami Dolphins' : 'MIA', 'Minnesota Vikings' : 'MIN', 
    'New England Patriots' : 'NWE', 'New Orleans Saints' : 'NOR', 
    'New York Giants' : 'NYG', 'New York Jets' : 'NYJ', 
    'Las Vegas Raiders' : 'LVR', 'Oakland Raiders' : 'LVR',
    'Philadelphia Eagles' : 'PHI', 
    'Pittsburgh Steelers' : 'PIT', 'San Francisco 49ers' : 'SFO', 
    'Seattle Seahawks' : 'SEA', 'Tampa Bay Buccaneers' : 'TAM', 
    'Tennessee Titans' : 'TEN', 'Washington Redskins' : 'WAS',
    'Washington Football Team': 'WAS'
    }

In [7]:


# Data processing



for season in season_dict:
    for team in season_dict[season]:
        iter_df = season_dict[season][team] # current iteration of the dataframe for season-team
        
        iter_df.rename(column_clean_dict, inplace=True, axis=1) # renames columns
        iter_df.drop('Unnamed: 4', axis=1, inplace=True) # drops an unused column
        
        iter_df.drop(iter_df[iter_df['week'] == 'Playoffs'].index, inplace=True, axis=0) # drops blank row separating playoffs from regular season
        iter_df.drop(iter_df[iter_df['opp'] == 'Bye Week'].index, inplace=True, axis=0) # drops bye weeks, will replace in future.
        iter_df.drop(iter_df[iter_df['date']== 'Playoffs'].index, inplace=True, axis=0) # some rows have playoff marker in date
        
        iter_df['opp'].replace(opp_dict, inplace = True)
        iter_df['team'].replace(acro_dict, inplace = True)
        
        iter_df['week'].fillna(value=0, inplace=True)
        iter_df['week'] = iter_df['week'].astype(str)
        iter_df['week'].replace({'Wild Card': 18, 'Division': 19, 'Conf. Champ.': 20, 'SuperBowl': 21}, inplace=True) # replaces str in 'Week' with playoff jgames like extension of regular season
        iter_df['week'] = iter_df['week'].astype(int) # turns back to int
        
        
        iter_df['ot'].fillna(value= 0, inplace=True) # replaces NaNs in the overtime column with 0
        iter_df['ot'] = iter_df['ot'].astype(str)
        iter_df['ot'].replace({'OT': 1}, inplace=True) # replaces 'OT' with 1 to signal overtime happened
        
        iter_df['W/L'].replace({'W' : 1, 'L': 0, 'T': 2}, inplace=True) # sets wins, losses, ties to numbers
        
        iter_df['home/away'].fillna(value='HOME', inplace=True) # fills in 'HOME' for home game
        iter_df['home/away'].replace({'@': 'AWAY', 'N':'NEUTRAL'}, inplace=True) # fills in 'AWAY' for away game
        
        iter_df['opp_pass_yards'].fillna(value=0, inplace=True) # two games have a team with net 0 pass yeards
        iter_df['team_pass_yards'].fillna(value=0, inplace=True)
        
        iter_df['team_turnover'].fillna(value=0, inplace=True) # fills 0s in for NaNs
        iter_df['opp_turnover'].fillna(value=0, inplace=True) # fills 0s in for Nans
        
        iter_df['kickoff_time'].fillna(value='0', inplace=True)
        kick_time = []
        kick_split = []
        for i in iter_df['kickoff_time']:
            kick_time.append(i)
        for i in kick_time:
            kick_split.append(pd.to_datetime(i.split()[0]))
        iter_df['kickoff_time'] = kick_split
            
        
        month_list = []
        month_split = []
        for i in iter_df['date']:
            month_list.append(i)
        for i in month_list:
            month_split.append(i.split()[0])
        iter_df['month'] = month_split
        iter_df['month'].replace(month_dict, inplace=True)

        day_split = []
        for i in month_list:
            day_split.append(i.split()[1])
        iter_df['day'] = day_split
        iter_df['day'] = iter_df['day'].astype(int)
        
        year_list = []
        month_list = []
        day_list = []
        datetime_list = []

        for i in iter_df['year']:
            year_list.append(i)
        for i in iter_df['month']:
            month_list.append(i)
        for i in iter_df['day']:
            day_list.append(i)

        counter = 0
        for i in iter_df['year']:
            if month_list[counter] in [1, 2]:
                datetime_list.append(dt.datetime(year_list[counter] + 1, month_list[counter], day_list[counter]))
                
            else:
                datetime_list.append(dt.datetime(year_list[counter], month_list[counter], day_list[counter]))
            counter += 1
        iter_df['dt_date'] = datetime_list
        
        #Creates datetime column
        counter = 0
        dt_list = []
        for i in iter_df['dt_date']:
            dt_list.append(dt.datetime.combine(datetime_list[counter], kick_split[counter].time()))
            counter += 1
        iter_df['datetime'] = dt_list
        
        # drops games that haven't been played yet
        
        #iter_df.drop(iter_df[iter_df['datetime'] > dt.datetime.today()].index, axis=0, inplace=True)
        
        
season_dict[2020]['chi'].head()

C:\Users\fitzp\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,week,day_week,date,kickoff_time,W/L,ot,record,home/away,opp,team_score,...,offense_expected_points,defense_expected_points,sp_expected_points,season,year,team,month,day,dt_date,datetime
0,1,Sun,September 13,2020-11-02 13:00:00,1.0,0,1-0,AWAY,DET,27.0,...,10.67,-7.96,0.12,2020,2020,CHI,9,13,2020-09-13,2020-09-13 13:00:00
1,2,Sun,September 20,2020-11-02 13:00:00,1.0,0,2-0,HOME,NYG,17.0,...,3.42,-0.28,-3.92,2020,2020,CHI,9,20,2020-09-20,2020-09-20 13:00:00
2,3,Sun,September 27,2020-11-02 13:00:00,1.0,0,3-0,AWAY,ATL,30.0,...,2.87,-5.82,5.24,2020,2020,CHI,9,27,2020-09-27,2020-09-27 13:00:00
3,4,Sun,October 4,2020-11-02 13:00:00,0.0,0,3-1,HOME,IND,11.0,...,-1.13,-2.28,-7.13,2020,2020,CHI,10,4,2020-10-04,2020-10-04 13:00:00
4,5,Thu,October 8,2020-11-02 20:20:00,1.0,0,4-1,HOME,TAM,20.0,...,-0.67,3.30,1.05,2020,2020,CHI,10,8,2020-10-08,2020-10-08 20:20:00


In [8]:
# exports each team's data as an individual sheet in case I want to come back and work with those on a focused level
# note that this bifucates the data, since the complete dataset requires a bit more processing to deal with duplicate rows. The duplication is not an issue on a by team basis

for team in teams:
    full_team = pd.DataFrame()
    for season in season_dict:
        full_team = full_team.append(season_dict[season][team])
        full_team = full_team[column_ord]
        full_team = full_team.reset_index()
        full_team.drop('index', axis = 1, inplace = True)
        
    full_team.to_csv(r'data\idv_team_data\{}.csv'.format(acro_dict[team]))

In [9]:
#  filters dupes in regular season, separates playoff games to remove dupes, then recombines to clean data set
raw_data = pd.DataFrame()
for season in season_dict:
    for team in teams:
        raw_data = raw_data.append(season_dict[season][team])
        raw_data = raw_data[column_ord]
        raw_data = raw_data.reset_index()
        raw_data.drop('index', axis = 1, inplace = True)
        
reg_data = raw_data[raw_data['home/away'] == 'HOME']

playoff_data = raw_data[raw_data['home/away'] == 'NEUTRAL']
playoff_data = playoff_data.reset_index()
playoff_data.drop('index', axis=1, inplace = True)
playoff_data.drop(range(0,36,2), axis = 0 , inplace= True)
playoff_data = playoff_data.reset_index()
playoff_data.drop('index', axis=1, inplace = True)

full_data = reg_data.append(playoff_data)
full_data = full_data.sort_values('season')
full_data = full_data.reset_index()
full_data.drop('index', axis=1, inplace = True)

        


In [10]:
# export of the full data set
full_data.to_csv(r'data\clean_data_set.csv', index=False)